In [51]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time

In [52]:
# function for plotting/visualization
def set_cartesian_plane():
    ax = plt.figure().add_subplot(1, 1, 1)
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.set_xlim([-1.5, 1.5])
    ax.set_ylim([-1.5, 1.5])

def generate_target_function():
    # generate two random points
    p1 = np.random.uniform(-1, 1, size=2)
    p2 = np.random.uniform(-1, 1, size=2)
    
    # Using points to get the parameters m and c for y = mx + c
    m = (p1[1] -p2[1])/(p1[0] - p2[0]) # m = (y1 - y2)/ (x1 -x2)
    c = p1[1] - m*p1[0] # c = y - mx   
    
    w_target = np.array([c, m, -1])
    
    return w_target

def run_linear_regression(X, y):
    pinv_X = np.dot(np.linalg.pinv(np.dot(X.T, X)), X.T)
    w = np.dot(pinv_X, y)# w = pseudo_inv(X)*y 
    return w

#This method is meant to return the pseudo inverse of the matrix X
def pseudo_inv(X):
    Xt = np.transpose(X)
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.inv(XtX)
    pinv_X = np.dot(XtX_inv, Xt)
    return pinv_X


# this method returns the same matrix with ones concatenated
def getInputX(x_vals):
    X = x_vals
#     X = X.reshape(1, X.size)
    X = np.transpose(X)
    ones = np.ones((1,N_points))
    X = np.concatenate((ones, X), axis = 0) # concatenate ones to account for bias term
    X = np.transpose(X)# X= vector containing points input x as its rows
    return X

def classify(point, w):
    return np.sign(np.dot(point, w))


In [53]:
'''QUESTION 5'''
N_experiments= 1000
N_points = 100
g_list = [] # keeps a list of all the final hypotheses
target_function_list = [] # keep a list of all of the target functions for linear classification
E_in_list = [] # keeps a list of all of the in sample errors of the LSRLs for classifying points



for i in range(N_experiments):
    w_target = generate_target_function()
    
    # generate specified number of random points
    x_n = np.random.uniform(-1, 1, size=(N_points, 2))

    #concantenate ones as the first column, bias term
    X = getInputX(x_n)
    
    # taking dot product of N x 3 point array and 3 x 1 weights array to get N x 1 classifications
    # this is basically checking the sign of c + mx -y. 
    # If this is positive, then point is under the line. ( y < mx + c)
    # If this is negative, then the point is above the line. (y > mx + c)
    y = np.sign(np.dot(X, w_target)) # ground truth classification
    
    # compute linear regression weights vector
    w = run_linear_regression(X, y)
    
    # classification predictions from linear regression
    y_lr = np.sign(np.dot(X, w))
    
    E_in = sum(y_lr != y) / N_points # calculating in sample error

    E_in_list.append(E_in)
    g_list.append(w)
    target_function_list.append(w_target)
    
E_in_avg = sum(E_in_list)/len(E_in_list)
print("Average in-sample error: ", E_in_avg)

Average in-sample error:  0.03917000000000009


In [43]:
'''
QUESTION 6
'''
N_experiments= 1000
N_test_points = 1000
E_out_list = []
for i, final_hyp in enumerate(g_list):
    target_function = target_function_list[i]
    x_test_points = np.concatenate((np.ones((N_test_points, 1)), 
                                    np.random.uniform(-1, 1, size=(N_test_points, 2))), 
                                    axis = 1)
    y_actual = np.sign(np.dot(x_test_points, target_function))
    y_predicted = np.sign(np.dot(x_test_points, final_hyp))
    
    E_out = sum(y_actual != y_predicted) / N_test_points # calculating out of sample error
    
    E_out_list.append(E_out)

E_out_avg = sum(E_out_list)/len(E_out_list)
print('Average out of sample error over 1000 trials: ', E_out_avg)

Average out of sample error over 1000 trials:  0.049127999999999984


In [60]:
'''
QUESTION 7
'''
N_points = 10
N_experiments = 1000
iterations_list = []
for i in range(N_experiments):
    # generating a random target function
    w_target = generate_target_function()
    
    # generating random points with bias term
    x_n = np.concatenate((np.ones((N_points, 1)), np.random.uniform(-1, 1, size=(N_points, 2))), axis = 1)
    
    #classifying according to target
    y_actual = np.sign(np.dot(x_n, w_target))
    
    # getting weights from linear regression, will serve as initial weights for perceptron learning algorithm(PLA)
    w_regression = run_linear_regression(x_n, y_actual)
    
    #
    y_predicted = np.sign(np.dot(x_n, w_regression))
    
    iterations = 0
    # the perceptron algorithm will run until all the points are classified correctly
    while not sum(y_predicted != y_actual) == 0:
        
        iterations += 1 # keep track of amount of iterations of PLA
        
        correctNs = [] # points correctly classified
        incorrectNs = [] # points misclassified
        
        for n, point in enumerate(x_n):
            if(y_predicted[n] == y_actual[n]):
                correctNs.append(n)
            else:
                incorrectNs.append(n)
        
        point_to_update = random.choice(incorrectNs)
        for n, weight in enumerate(w_regression):
            w_regression[n] += y_actual[point_to_update] * x_n[point_to_update][n]
        
        
        y_predicted = np.sign(np.dot(x_n, w_regression))

    iterations_list.append(iterations)

print("Average number of iterations to converge PLA using Linear Regression as initial weights: ", sum(iterations_list)/len(iterations_list))
    

Average number of iterations to converge PLA using Linear Regression as initial weights:  3.638
